# Midterm Project: Retail Sales Data Mart

## 📌 Business Process Overview
This project models a **retail sales system** where customers purchase products over time.  The goal is to analyze **sales trends, customer spending, and product performance** over time.

## 📌 Data Warehouse Design (`northwind_dw2`)
- **Fact Table:**
  - `fact_orders` (Tracks customer purchases, order details, and revenue)

- **Dimension Tables:**
  - `dim_products` (Describes products: name, category, pricing)
  - `dim_date` (Tracks order dates: year, month, day of the week)
  - `dim_customers` (Identifies customers: location, demographics)

## 📌 Questions We Can Answer:
- 💰 **Total sales per product?** (`fact_orders` + `dim_products`)
- 📆 **Orders per year?** (`fact_orders` + `dim_date`)
- 👥 **Top spending customers?** (`fact_orders` + `dim_customers`)


In [1]:
import os
import json
import certifi
import pandas as pd
import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

# to connect arguments (used from lab4)
mysql_args = {
    "uid": "root",
    "pwd": "Newyear2024!",
    "hostname": "localhost",
    "dbname": "northwind_dw2"
}

# MongoDB Connection Arguments
mongodb_args = {
    "cluster_location": "local",
    "db_name": "northwind_dw2_mongo"
}

# gets data from sql
def get_sql_dataframe(sql_query, **args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    dframe = pd.read_sql(text(sql_query), connection)
    connection.close()
    
    return dframe

# inserts the data in sql 
def set_dataframe(df, table_name, pk_column, db_operation, **args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')

    connection.close()

# gets data from MongoDB
def get_mongo_dataframe(mongo_client, db_name, collection, query):
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe

# insert data into MongoDB
def set_mongo_collections(mongo_client, db_name, collection_name, json_data):
    db = mongo_client[db_name]
    db.drop_collection(collection_name)
    
    collection = db[collection_name]
    collection.insert_many(json_data)
    mongo_client.close()


In [2]:
# extract `dim_date` 
sql_dim_date = "SELECT * FROM northwind_dw2.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
display(df_dim_date.head())


,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
2,20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
3,20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
4,20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


In [3]:
# extract `dim_products`
sql_dim_products = "SELECT * FROM northwind_dw2.dim_products;"
df_dim_products = get_sql_dataframe(sql_dim_products, **mysql_args)
display(df_dim_products.head())

,product_key,product_id,product_code,product_name,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category
0,1,1,NWTB-1,Northwind Traders Chai,13.5000,18.00,10,40,10 boxes x 20 bags,0,10.0,Beverages
1,2,3,NWTCO-3,Northwind Traders Syrup,7.5000,10.00,25,100,12 - 550 ml bottles,0,25.0,Condiments
2,3,4,NWTCO-4,Northwind Traders Cajun Seasoning,16.5000,22.00,10,40,48 - 6 oz jars,0,10.0,Condiments
3,4,5,NWTO-5,Northwind Traders Olive Oil,16.0125,21.35,10,40,36 boxes,0,10.0,Oil
4,5,6,NWTJP-6,Northwind Traders Boysenberry Spread,18.7500,25.00,25,100,12 - 8 oz jars,0,25.0,"Jams, Preserves"


In [4]:

# checks to see if there are duplicate columns
duplicate_columns = df_dim_products.columns[df_dim_products.columns.duplicated()].tolist()
print("Duplicate Columns:", duplicate_columns)

# renames any duplicate columns if needed
df_dim_products = df_dim_products.loc[:, ~df_dim_products.columns.duplicated()]

# Display first few rows after removing duplicates
display(df_dim_products.head())


Duplicate Columns: []


,product_key,product_id,product_code,product_name,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category
0,1,1,NWTB-1,Northwind Traders Chai,13.5000,18.00,10,40,10 boxes x 20 bags,0,10.0,Beverages
1,2,3,NWTCO-3,Northwind Traders Syrup,7.5000,10.00,25,100,12 - 550 ml bottles,0,25.0,Condiments
2,3,4,NWTCO-4,Northwind Traders Cajun Seasoning,16.5000,22.00,10,40,48 - 6 oz jars,0,10.0,Condiments
3,4,5,NWTO-5,Northwind Traders Olive Oil,16.0125,21.35,10,40,36 boxes,0,10.0,Oil
4,5,6,NWTJP-6,Northwind Traders Boysenberry Spread,18.7500,25.00,25,100,12 - 8 oz jars,0,25.0,"Jams, Preserves"


In [5]:
# rename `product_id` to `product_key`
df_dim_products.rename(columns={"product_id": "product_key"}, inplace=True)

# fill missing values in `category` with "Unknown"
df_dim_products.fillna({"category": "Unknown"}, inplace=True)

# keep only relevant columns for MongoDB
df_dim_products = df_dim_products[["product_key", "product_name", "category", "list_price"]]

# display first few rows after transformation
display(df_dim_products.head())


,product_key,product_key,product_name,category,list_price
0,1,1,Northwind Traders Chai,Beverages,18.00
1,2,3,Northwind Traders Syrup,Condiments,10.00
2,3,4,Northwind Traders Cajun Seasoning,Condiments,22.00
3,4,5,Northwind Traders Olive Oil,Oil,21.35
4,5,6,Northwind Traders Boysenberry Spread,"Jams, Preserves",25.00


In [6]:
# convert to JSON format
products_json = df_dim_products.to_dict(orient="records")

# insert into MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
set_mongo_collections(client, mongodb_args["db_name"], "dim_products", products_json)

/var/folders/pm/fr4p6g5x33s7jjzlf3w9_hpm0000gn/T/ipykernel_11400/1178894821.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  products_json = df_dim_products.to_dict(orient="records")


In [7]:
# Extract `dim_customers`
sql_dim_customers = "SELECT * FROM northwind_dw2.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_dim_customers, **mysql_args)

#display the result
display(df_dim_customers.head())


,customer_key,customer_id,company,last_name,first_name,job_title,business_phone,fax_number,address,city,state_province,zip_postal_code,country_region
0,1,1,Company A,Bedecs,Anna,Owner,(123)555-0100,(123)555-0101,123 1st Street,Seattle,WA,99999,USA
1,2,2,Company B,Gratacos Solsona,Antonio,Owner,(123)555-0100,(123)555-0101,123 2nd Street,Boston,MA,99999,USA
2,3,3,Company C,Axen,Thomas,Purchasing Representative,(123)555-0100,(123)555-0101,123 3rd Street,Los Angelas,CA,99999,USA
3,4,4,Company D,Lee,Christina,Purchasing Manager,(123)555-0100,(123)555-0101,123 4th Street,New York,NY,99999,USA
4,5,5,Company E,O’Donnell,Martin,Owner,(123)555-0100,(123)555-0101,123 5th Street,Minneapolis,MN,99999,USA


In [8]:
# convert to JSON and insert into mongoDB
customers_json = df_dim_customers.to_dict(orient="records")

#insert to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
set_mongo_collections(client, mongodb_args["db_name"], "dim_customers", customers_json)

In [9]:
# extract `fact_orders`
sql_fact_orders = "SELECT * FROM northwind_dw2.fact_orders;"
df_fact_orders = get_sql_dataframe(sql_fact_orders, **mysql_args)

# save to CSV
df_fact_orders.to_csv("fact_orders.csv", index=False)



In [10]:
# read csv
df_fact_orders = pd.read_csv("fact_orders.csv")

# drop any duplicate order IDs but keep the first 
df_fact_orders = df_fact_orders.drop_duplicates(subset=["order_id"], keep="first")

#display the first few rows
display(df_fact_orders.head())

# load into MySQL 
set_dataframe(df_fact_orders, "fact_orders", "order_id", "replace", **mysql_args)


,fact_order_key,order_id,customer_key,employee_key,shipper_key,product_key,order_date_key,quantity,unit_price,order_status,order_details_status
0,1,30,27,9,2.0,13,20060115,100.0,14.0,Closed,Invoiced
2,3,31,4,3,1.0,6,20060120,10.0,30.0,Closed,Invoiced
5,6,32,12,4,2.0,1,20060122,15.0,18.0,Closed,Invoiced
7,8,33,8,6,3.0,10,20060130,30.0,9.2,Closed,Invoiced
8,9,34,4,9,3.0,10,20060206,20.0,9.2,Closed,Invoiced


In [11]:
# get total orders per year
sql_yearly_orders = """
SELECT d.calendar_year, COUNT(o.order_id) AS total_orders
FROM northwind_dw2.fact_orders o
JOIN northwind_dw2.dim_date d ON o.order_date_key = d.date_key
GROUP BY d.calendar_year;
"""
df_yearly_orders = get_sql_dataframe(sql_yearly_orders, **mysql_args)
df_yearly_orders


,calendar_year,total_orders
0,2006,58


In [12]:
# get total sales per product
sql_product_sales = """
SELECT p.product_name, SUM(o.unit_price * o.quantity) AS total_revenue
FROM northwind_dw2.fact_orders o
JOIN northwind_dw2.dim_products p ON o.product_key = p.product_key
GROUP BY p.product_name;
"""
df_product_sales = get_sql_dataframe(sql_product_sales, **mysql_args)
df_product_sales

,product_name,total_revenue
0,Northwind Traders Beer,6818.00
1,Northwind Traders Dried Plums,262.50
2,Northwind Traders Dried Pears,1200.00
3,Northwind Traders Dried Apples,2120.00
4,Northwind Traders Chai,720.00
5,Northwind Traders Coffee,29900.00
6,Northwind Traders Chocolate Biscuits Mix,782.00
7,Northwind Traders Chocolate,2550.00
8,Northwind Traders Clam Chowder,2798.50
9,Northwind Traders Curry Sauce,2600.00


In [15]:
# groups order based on company names
sql_customer_sales = """
SELECT c.company AS customer_company, 
       SUM(o.unit_price * o.quantity) AS total_spent
FROM northwind_dw2.fact_orders o
JOIN northwind_dw2.dim_customers c ON o.customer_key = c.customer_key
GROUP BY customer_company
ORDER BY total_spent DESC;
"""
df_customer_sales = get_sql_dataframe(sql_customer_sales, **mysql_args)
df_customer_sales


,customer_company,total_spent
0,Company BB,15432.50
1,Company G,13800.00
2,Company F,8007.50
3,Company D,4949.00
4,Company H,4683.00
5,Company I,3786.50
6,Company Z,3625.25
7,Company CC,2905.50
8,Company C,2550.00
9,Company A,2410.75
